# Algology Data

In this data-set our goal is observing how beneficial procedure was for the patients.

### Inserting Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import re
# import warnings
# warnings.simplefilter('ignore')

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

### Inserting Data

In [2]:
data = pd.read_excel("2018 algoloji işlemler(Original).xlsx", sheet_name='ocak-nisan 2018')

### To find unique values of cells have more than one value (seperated by '+' | ',' | '/')

In [3]:
columns = ['Diagnosis', 'R/L', 'Procedure', 'Approach']  # columns that include typos
unique_values = {}  # keys, column name: values, unique values have typos
for column in columns:
    for i in np.unique(data[column].astype('str')):
        unique_values.setdefault(column, [])
        if "+" in i or "/" in i or "," in i:
            unique_values[column].append(i)
splitted_unique_vals = {}  # keys, typos: values, splitted values which typos fixed
for key, values in unique_values.items():
    for val in values:
        splitted_val = re.split('\+|,|/',val)
        for idx in range(len(splitted_val)):
            splitted_val[idx] = splitted_val[idx].strip()
        splitted_unique_vals[val] = splitted_val

### Calculating how many row should belong to same individual with different types

Length keeps the number of new samples for one row.
We multiply each cell length because each row may have more than one cell includes typos.

In [4]:
total_counts = {} # keys, row number: values, counts of how many row belongs to same sample
counter = -1
for row in data.values:
    counter += 1
    length = 1
    for key, value in unique_values.items():
        for cell in value:
            if cell in row:
                length *= len(splitted_unique_vals[cell])
    total_counts[counter] = length

### To get new edited data with new samples 

In [5]:
edited_array = []
for row, row_num in zip(data.values, sorted(total_counts)):
    new_rows = []
    if total_counts[row_num] == 1:
        edited_array.append(row)
        continue
    for len_ in range(total_counts[row_num]):
        new_rows.append(row)
    for cell in row:
        if cell in splitted_unique_vals:
            part = len(new_rows)//len(splitted_unique_vals[cell])
            start = 0
            end = part
            for len_ in range(len(splitted_unique_vals[cell])):
                for i in range(start,end):
                    new_rows[i] = np.where(new_rows[i]==cell, splitted_unique_vals[cell][len_], new_rows[i])
                start = end
                end += part
    edited_array.extend(new_rows)
edited_array = np.array(edited_array)

In [6]:
edited_data = pd.DataFrame(edited_array, columns = data.columns)
# edited_data.to_excel("2018 algoloji işlemler(Edited by Huzeyfe).xlsx")